# VirusTotal Search to Iris Investigate

## Research Questions

Is there value in taking a VirusTotal search, pulling all of the related domains and IPs from it, then loading them into Iris Investigate?

Do pivots then on those initial data points provide more useful information for searches?

---

# Setup

Import packages, then load in our credentials using dotenv which allows us to use a `.env` file in the Notebooks directory. This makes sure we don't store our credentials in a notebook, but also that they do not rely on running on a single developer's machine. We can have Jupyter Lab launched via Docker or via CI/CD and load those environment variables in automatically.

In [ ]:
import os
import pprint

import domaintools
import dotenv
import matplotlib
import matplotlib.pyplot as plt
import requests

In [ ]:
dotenv.load_dotenv()
vt_api_key = os.getenv('VT_API_KEY')
dt_api_user = os.getenv('DT_API_USER')
dt_api_key = os.getenv('DT_API_KEY')

# Query

Initial query to grab all of the objects that match our search.

In [ ]:
vt_api_url_base = 'https://www.virustotal.com/api/v3'
headers = {'x-apikey': vt_api_key}

query = 'tag:cve-2020-0796'

results = requests.get('{}/intelligence/search?query={}&limit=300'.format(vt_api_url_base, query), headers=headers)

# Extract Links to Objects

From our query, extract all of the links that lead to the objects we want to observe. We only want things that have shown up as malicious or suspicious here. From this we can investigate all of those objects and hunt for relationships we want to extract from them.

In [ ]:
links = list()
for result in results.json()['data']:
    malicious = result['attributes']['last_analysis_stats']['malicious']
    suspicious = result['attributes']['last_analysis_stats']['suspicious']
    if malicious > 0 or suspicious > 0:
        links.append(result['links']['self'])

# Extract Relationships

Go through each objects, extract the relationships, and set them up for enhancement via Iris Investigate.

In [ ]:
traffic_data = list()
for link in links:
    ip_traffic = list()
    dns_lookups = list()
    request = requests.get('{}/behaviours'.format(link), headers=headers)
    
    if len(request.json()['data']) == 0:
        continue
        
    data = request.json()['data']

    for instance in data:
        ip_traffic.extend(instance['attributes'].get('ip_traffic', list()))
        dns_lookups.extend(instance['attributes'].get('dns_lookups', list()))
        
    unique_ips = {traffic['destination_ip'] for traffic in ip_traffic}
    unique_hostnames = {lookup['hostname'] for lookup in dns_lookups}
        
    traffic_data.append({'ip_traffic': ip_traffic,
                         'dns_lookups': dns_lookups,
                         'url': link})

# Query IPs with Iris Investigate

Cycle through all of this and now enrich with Iris Investigate, extract just the domains for some other work, and print domains in a copy-paste-able into Iris if needed.

In [ ]:
dt_api = domaintools.API(dt_api_user, dt_api_key)
domain_data = list()

for traffic in traffic_data:
    for ip_traffic in traffic.get('ip_traffic'):
        domain_data.extend([domain for domain in dt_api.iris_investigate(ip=ip_traffic['destination_ip'])])
        
domains = {domain['domain'] for domain in domain_data}

# If you want to copy and paste this into Iris for an investigation, uncomment:
# print(','.join(domains))

# Risk Score Graph

Extract our risk score from enriched data, then graph that to see levels of badness here. If it is rated at 100 it is already on a blocklist.

In [ ]:
risk_scores = [0] * 100
for domain in domain_data:
    if domain.get('domain_risk') == None or domain['domain_risk'].get('risk_score') == None:
        print('No risk score for {}.'.format(domain['domain']))
        continue
        
    risk_scores[domain['domain_risk']['risk_score'] - 1] += 1

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(100), risk_scores)
ax.set(xlabel='Risk Score 1 to 100', ylabel='Count of Domains',
       title='Plotted Risk Scores of {} Domains'.format(len(domains)))
ax.grid()
# If you want to save the figure for later versus just show in-line.
# fig.savefig('filename.png')
plt.show()